# Introduction

## Problem

About a quarter million people are moving to Colorado every year. Many of these people are looking to set up new business and may not know what city they would want to set up their business. Specifically where in each city. This is important so people who are starting up their business are in a great location that will make them successful.

### Data

The data I will be using are zip codes of all the cities in colorado with a population count. I will be joining these with other zip code data sets that are specific to cities. I will be using this data to find optimal cities to build a specific business in and what areas in those cities are best. I will be using foursquare to find information on each cities and see what kind of business are in each zip codes in order to provide feedback on what businesses would be ideal in each area. 